In [ ]:
data_train, data_test, _ = preprocessing(df = df[:int(len(df)*0.2)], lag = 1, sequence_length=100, dif_all=True, train_size=0.9 )
data_module = StockPriceDataModule(train_sequence = data_train,  test_sequence= data_test, batch_size=64, num_workers = 4)
data_module.setup()

for item in data_module.train_dataloader():
    print(item.keys())
    print(item['sequence'].shape)
    print(item['label'].shape)
    break


In [ ]:
checkpoint_callback = ModelCheckpoint(
    dirpath="checkpoints",
    filename="best_checkpoint3",
    save_top_k=1,
    verbose=True,
    monitor="train_loss",
    mode="min"
)

#logger = TensorBoardLogger("logs/", name="lightninglstm")

early_stopping_callback = EarlyStopping(monitor="val_loss", patience=2)

progressbar = TQDMProgressBar(refresh_rate=100)

In [ ]:
model = LightningLSTM(input_size=7, hidden_size = 100, num_layers = 3)
trainer = pl.Trainer(max_epochs=15, callbacks=[checkpoint_callback,progressbar], accelerator="gpu", devices=1, limit_val_batches=0)
trainer.fit(model, data_module)

In [ ]:
from Evaluation import evaluation_plots
from tqdm import tqdm
df_test = data_test[:int(len(data_test[0])*0.1)]
print(len(df_test))
test = StockDataset(df_test)

predictions = []
labels = []

pbar = tqdm(total=len(test))

for item in test:
  
  pbar.update(1)
  sequence = item["sequence"]
  label = item['label']

  _, output = model(sequence.unsqueeze(dim=0))
  predictions.append(float(output.detach().numpy()[0]))
  labels.append(float(label.detach().numpy()))

pbar.close()

evaluation_plots.plot_predictions_vs_labels(predictions, labels)